# Data Dictionary
#### DATE_TIME 
It is the field that contains date and time information. Data format YYYY-MM-DD HH24: MI: SS format. The date break is hourly.
#### OBSERVATORY_NAME
It indicates the type number of the sensor where the measurement is made.
#### AVERAGE_TEMPERATURE
Average humidity (% RH) measured from the respective sensor in the given hour.
#### AVERAGE_HUMIDITY
Average wind speed (km / h) measured from the relevant sensor for the given hour.
#### AVERAGE_WIND
Average wind direction (km / h) measured from the respective sensor at the given hour.
#### AVERAGE_DIRECTIONOFWIND
Average precipitation amount (kg / m²) measured from the relevant sensor in the given hour.
#### AVERAGE_PRECIPITATION
Average amount of precipitation (kg / m²) measured from the relevant sensor in the given hour.
#### AVERAGE_ROAD_TEMPERATURE
Average road temperature (& ordm; C) measured from the respective sensor at the given hour.

In [333]:
import os
import pandas as pd
import numpy as np

import locale
from locale import atof
import xlsxwriter

from PreProcessingUtil import preprocessing

In [316]:
# Select rows in a DataFrame between two dates
def selectedDateFrame(dfPol, dfMet):
    dfPol = dfPol[(startDate<=dfPol['Tarih']) & (dfPol['Tarih']<endDate)]
    dfMet = dfMet[(startDate<=dfMet['DATE_TIME']) & (dfMet['DATE_TIME']<endDate)]
    return dfPol, dfMet

# Function to fill missing rows
def fillingEmptyRows(dfTrue,dfFalse):
    for x in range (dfTrue.shape[0]):
        if not (str(dfTrue['Tarih'].iloc[x])[:13] == str(dfFalse['DATE_TIME'].iloc[x])[:13]):
            temp = dfFalse.iloc[x-1] # get previous value
            temp['DATE_TIME'] = dfTrue['Tarih'].iloc[x] # set true date
            dfFalse = Insert_row(x, dfFalse, temp) # insert missing value        
    return dfTrue, dfFalse

# Function to insert row in the dataframe
def Insert_row(row_number, df, row_value):
    # Starting value of upper half
    start_upper = 0
   
    # End value of upper half
    end_upper = row_number
   
    # Start value of lower half
    start_lower = row_number
   
    # End value of lower half
    end_lower = df.shape[0]
   
    # Create a list of upper_half index
    upper_half = [*range(start_upper, end_upper, 1)]
   
    # Create a list of lower_half index
    lower_half = [*range(start_lower, end_lower, 1)]
   
    # Increment the value of lower half by 1
    lower_half = [x.__add__(1) for x in lower_half]
   
    # Combine the two lists
    index_ = upper_half + lower_half
   
    # Update the index of the dataframe
    df.index = index_
   
    # Insert a row at the end
    df.loc[row_number] = row_value
    
    # Sort the index labels
    df = df.sort_index()
   
    # return the dataframe
    return df

In [27]:
# creating dataframes per town
df1  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202001.csv"))
df2  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202002.csv"))
df3  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202003.csv"))
df4  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202004.csv"))
df5  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202005.csv"))
df6  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202006.csv"))
df7  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202007.csv"))
df8  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202008.csv"))
df9  = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202009.csv"))
df10 = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202010.csv"))
df11 = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202011.csv"))
df12 = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202012.csv"))
df13 = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202101.csv"))
df14 = pd.read_csv(os.path.join("../", "datasets", "meteorology_observation", "meteorology_observation_202102.csv"))

In [302]:
# concat all dataframes
bigdata = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14], ignore_index=True, sort=False)

In [303]:
# distinct town names
bigdata['OBSERVATORY_NAME'].unique()

array(['BEYKOZ_ANADOLU_FENERI', 'ISTOC_MAHMUTBEY',
       'GUNGOREN_DAVUTPASA_MARMARA', 'KILYOS', 'SILE_DARLIK', 'PASAKOY',
       'SANCAKTEPE_MGM', 'ESENLER', 'HACIOSMAN_SARIYER', 'SILE_ISAKOY',
       'DURUSU', 'SUBASI', 'CAMLICA_TUNEL_K', 'BAYRAMOGLU_TUZLA',
       'KAMILOBA', 'CAMLICA_LIBADIYE', 'SARIYER_ITU_MASLAK', 'CATALCA',
       'BUYUKADA', 'SABIHAGOKCEN', 'EMINONU', 'OLIMPIYAT', 'G_O_PASA',
       'BESIKTAS_YILDIZ', 'SILIVRI_ORMAN_SAHASI', 'USKUDAR_MGM', 'SILE',
       'RIVA_TUNEL_G', 'BUYUKCEKMECE_MGM', 'KARTAL_AYDOS_DAGI',
       'UMRANIYE', 'ODAYERI', 'ARNAVUTKOY_MGM', 'HADIMKOY', 'BEYKOZ',
       'SUREYYAPASA', 'SAMANDIRA', 'SARIYER', 'AKOM', 'CATALCA_MGM',
       'B_CEKMECE_SVIRAJLARI', 'BAHCESEHIR_I_KULE', 'SILIVRI_MGM',
       'TERKOS', 'CAVUSBASI', 'CANTA', 'TERKOS_BARAJI', 'PENDIK',
       'CEKMEKOY_OMERLI_MGM', 'SISLİ_MGM', 'KARTAL', 'CIFTALAN', 'SILE_2',
       'KADIKOY_GOZTEPE_MGM', 'GOZTEPE', 'BEYLİKDUZU_MGM', 'YSS_KOPRUSU',
       'BEYKOZ_MGM', 'AHL_BAKIRKOY', 

In [341]:
# selected columns
reducedData = bigdata[["DATE_TIME", "OBSERVATORY_NAME", "AVERAGE_TEMPERATURE", "AVERAGE_HUMIDITY", "AVERAGE_WIND", "AVERAGE_DIRECTIONOFWIND", "AVERAGE_PRECIPITATION", "AVERAGE_ROAD_TEMPERATURE"]]

In [342]:
# town names different in meteorology and pollutant datasets, so find and give the convenient
polTownName = 'kadıkoy'
metTownName = 'KADIKOY_GOZTEPE_MGM'

In [343]:
# all pollutant values according to a particular town (sorted by date time)
polPath = polTownName + '.xlsx'
dfPollutant = pd.read_excel(os.path.join("../", "datasets", "pollutants", polPath),engine='openpyxl',parse_dates=True,thousands='.')

In [344]:
# all meteorology values according to a particular town (sorted by date time)
dfMeteorology = reducedData[reducedData['OBSERVATORY_NAME']=='KADIKOY_GOZTEPE_MGM'].sort_values('DATE_TIME')

In [345]:
# determining dates
startDate = '2020-01-01 00:00:00'
endDate = '2021-02-21 00:00:00'

In [346]:
# pooling selected rows according to determined date time
dfPol, dfMet = selectedDateFrame(dfPollutant, dfMeteorology)

In [347]:
# filling the missing rows with previous row value
dfPol, dfMet = fillingEmptyRows(dfPol, dfMet)

ipykernel_launcher:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/deneme/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [348]:
# making preprocessing (fixing values, filling empty rows)
dfPol = preprocessing(dfPol)

In [349]:
# reset index values
dfMet = dfMet.reset_index(drop=True)
dfPol = dfPol.reset_index(drop=True)

In [350]:
# concat dataframes
result = pd.concat([dfPol, dfMet], axis=1, join='inner')
display(result)

,Tarih,PM10 ( µg/m3 ),SO2 ( µg/m3 ),CO ( µg/m3 ),NO2 ( µg/m3 ),NOX ( µg/m3 ),O3 ( µg/m3 ),PM 2.5 ( µg/m3 ),DATE_TIME,OBSERVATORY_NAME,AVERAGE_TEMPERATURE,AVERAGE_HUMIDITY,AVERAGE_WIND,AVERAGE_DIRECTIONOFWIND,AVERAGE_PRECIPITATION,AVERAGE_ROAD_TEMPERATURE
0,2020-01-01 00:00:56,36.599998,5.2,986.700012,33.599998,965.200012,23.500000,20.200001,2020-01-01 00:00:56,KADIKOY_GOZTEPE_MGM,2.89216,66.98039,1.72549,38.52941,0.0,-99.0
1,2020-01-01 01:00:56,35.599998,5.2,848.500000,46.400002,861.900024,0.400000,20.000000,2020-01-01 01:00:56,KADIKOY_GOZTEPE_MGM,2.89216,66.98039,1.72549,38.52941,0.0,-99.0
2,2020-01-01 02:00:56,31.600000,5.9,827.200012,50.799999,819.000000,1.600000,18.500000,2020-01-01 02:00:56,KADIKOY_GOZTEPE_MGM,2.89216,66.98039,1.72549,38.52941,0.0,-99.0
3,2020-01-01 03:00:56,27.299999,6.0,785.200012,74.000000,716.200012,23.700001,17.500000,2020-01-01 03:00:56,KADIKOY_GOZTEPE_MGM,2.89216,66.98039,1.72549,38.52941,0.0,-99.0
4,2020-01-01 04:00:56,27.200001,5.9,872.000000,57.700001,744.799988,18.000000,18.500000,2020-01-01 04:00:56,KADIKOY_GOZTEPE_MGM,2.89216,66.98039,1.72549,38.52941,0.0,-99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10003,2021-02-20 19:00:56,53.799999,3.5,590.599976,81.800003,95.099998,6.800000,11.600000,2021-02-20 19:00:00,KADIKOY_GOZTEPE_MGM,3.17083,60.35417,1.65417,60.66667,0.0,-99.0
10004,2021-02-20 20:00:56,57.400002,3.8,769.299988,112.099998,110.599998,5.000000,15.200000,2021-02-20 20:00:00,KADIKOY_GOZTEPE_MGM,2.99825,62.35088,1.71053,68.17544,0.0,-99.0
10005,2021-02-20 21:00:56,55.299999,3.7,1327.599976,109.900002,110.699997,3.800000,16.000000,2021-02-20 21:00:00,KADIKOY_GOZTEPE_MGM,3.00364,65.07273,2.10182,51.74545,0.0,-99.0
10006,2021-02-20 22:00:56,55.000000,3.8,1138.000000,131.399994,114.000000,3.800000,11.500000,2021-02-20 22:00:00,KADIKOY_GOZTEPE_MGM,3.01455,66.00000,1.71273,49.61818,0.0,-99.0


In [351]:
# create excel writer object
path = '../datasets/training/'+polTownName+'_combined.xlsx'
writer = pd.ExcelWriter(path)
# write dataframe to excel
result.to_excel(writer)
# save the excel
writer.save()